## Создаем подключение к базе данных с помощью библиотеки sqlite3

In [62]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../data/checking-logs.sqlite')

## Создать новую таблицу datamart в базе данных, объединив таблицы pageviews и checker, используя только один запрос
- таблица должна иметь следующие столбцы: uid, labname, first_commit_ts, first_view_ts
- first_commit_ts — это просто новое имя столбца timestamp из таблицы проверки, оно показывает первый коммит из конкретной лаборатории и от конкретного пользователя.
- first_view_ts — первый визит пользователя в таблицу pageviews, временная метка, когда пользователь посетил ленту новостей
- статус = «готов» все еще должен быть фильтром
- numTrials = 1 все равно должен быть фильтром
- имена лабораторий должны быть из списка: 'laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1'
- таблица должна содержать только пользователей (uid с user_*), а не администраторов
- first_commit_ts и first_view_ts следует анализировать как datetime64[ns]

In [63]:
conn.execute("DROP TABLE IF EXISTS datamart")

query = '''
CREATE TABLE datamart AS
SELECT
    c.uid,
    c.labname,
    c.timestamp AS first_commit_ts,
    MIN(p.datetime) AS first_view_ts
FROM
    checker c
LEFT JOIN
    pageviews p ON c.uid = p.uid
WHERE
    c.status = 'ready'
    AND c.numTrials = 1
    AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND c.uid LIKE 'user_%'
GROUP BY
    c.uid, c.labname, c.timestamp
'''

conn.execute(query)

In [64]:
datamart = pd.read_sql_query("SELECT * FROM datamart", conn)

# Преобразование столбцов в datetime64[ns]
datamart['first_commit_ts'] = pd.to_datetime(datamart['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(datamart['first_view_ts'])

print(datamart)

        uid   labname            first_commit_ts              first_view_ts
0    user_1    laba04 2020-04-26 17:06:18.462708 2020-04-26 21:53:59.624136
1    user_1   laba04s 2020-04-26 17:12:11.843671 2020-04-26 21:53:59.624136
2    user_1    laba05 2020-05-02 19:15:18.540185 2020-04-26 21:53:59.624136
3    user_1    laba06 2020-05-17 16:26:35.268534 2020-04-26 21:53:59.624136
4    user_1   laba06s 2020-05-20 12:23:37.289724 2020-04-26 21:53:59.624136
..      ...       ...                        ...                        ...
135  user_8   laba04s 2020-04-19 10:22:35.761944                        NaT
136  user_8    laba05 2020-05-02 13:28:07.705193                        NaT
137  user_8    laba06 2020-05-16 17:56:15.755553                        NaT
138  user_8   laba06s 2020-05-16 20:01:07.900727                        NaT
139  user_8  project1 2020-05-14 15:42:04.002981                        NaT

[140 rows x 4 columns]


## Используя методы Pandas, создайте два фрейма данных: тестовый и контрольный.
- тест должен иметь пользователей, которые имеют значения в first_view_ts
- элемент управления должен иметь пользователей, у которых отсутствуют значения в first_view_ts
- замените отсутствующие значения в элементе управления средним значением first_view_ts тестовых пользователей, мы будем использовать это значение для будущего анализа
- сохраните обе таблицы в базе данных, они вам понадобятся в следующих упражнениях.

In [65]:
test = datamart.dropna(subset=['first_view_ts']) # Создание тестового фрейма данных

control = datamart[datamart['first_view_ts'].isna()] # Создание контрольного фрейма данных

# Замена отсутствующих значений в контрольном фрейме данных
mean_first_view_ts = test['first_view_ts'].mean()
control.loc[:, 'first_view_ts'] = control['first_view_ts'].fillna(mean_first_view_ts)

# Печать количества строк в datamart_df
print(f"Количество строк в datamart: {len(datamart)}")

# Печать количества строк в test_d
print(f"Количество строк в test: {len(test)}")
print(f"Количество строк в control: {len(control)}")

test.to_sql('test_datamart', conn, index=False, if_exists='replace') # Сохранение тестового фрейма данных в базе данных

control.to_sql('control_datamart', conn, index=False, if_exists='replace') # Сохранение контрольного фрейма данных в базе данных


Количество строк в datamart: 140
Количество строк в test: 59
Количество строк в control: 81


81

## Закрыть соединение

In [69]:
conn.close()